# sumaryzacja tekstu

In [8]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [9]:
!nvidia-smi

Sun Nov 16 20:07:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
# 1. Force uninstall both to clear the conflict
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.11.3
Uninstalling unsloth-2025.11.3:
  Successfully uninstalled unsloth-2025.11.3
Found existing installation: unsloth_zoo 2025.11.4
Uninstalling unsloth_zoo-2025.11.4:
  Successfully uninstalled unsloth_zoo-2025.11.4
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-tr8dqsot/unsloth-zoo_728b62ea2d174ba3963dbe14f0753c34
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-tr8dqsot/unsloth-zoo_728b62ea2d174ba3963dbe14f0753c34
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit b60fd19e4e1c27014553f7194669d66cdb96db2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 17.1 MB/s eta 0:00:00
  Created wheel for unsloth_zoo: filename=unsloth_zoo-2025.11.4-py3-none-any.whl size=281896 sha256=6ecd99aacb2284396

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-n66b9_wp/unsloth_de7dda2029ce4f3aaa9bff43c5669cbd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-n66b9_wp/unsloth_de7dda2029ce4f3aaa9bff43c5669cbd
  Resolved https://github.com/unslothai/unsloth.git to commit 38bdbed0bf65230bf076fd7e59583006348bde47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.11.3-py3-none-any.whl size=357217 sha256=a15f31dc37149a9cb41222cae9b9b9b26c253804ae2e7c8782afad681f34270c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3cv68qkb/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [11]:
!pip install "datasets<3.0.0"


  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.3.0
    Uninstalling datasets-4.3.0:
      Successfully uninstalled datasets-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.24.0 requires datasets>=3.0.0, but you have datasets 2.21.0 which is incompatible.
unsloth-zoo 2025.11.4 requires datasets!=4.0.*,!=4.1.0,<4.4.0,>=3.4.1, but you have datasets 2.21.0 which is incompatible.


In [12]:
!pip install bert-extractive-summarizer

In [13]:
!pip install pytorch-ignite
!pip install torchmetrics

## importowanie datesetu

In [14]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11332 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [15]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [16]:
from summarizer import Summarizer
model = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [18]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [19]:
num_workers = 1
batch_size = 1

In [20]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [21]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None, promptsuff=None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}{promptsuff}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        do_sample=True,
        top_k=0,
        temperature=0.5,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    for i, pred in enumerate(predictions):
        if pred.strip() == "":
            print("EMPTY PREDICTION")
            print(batch['document'][i])
    return bertscore(predictions, batch["summary"])

In [22]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [23]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [24]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None, promptsuff = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt, promptsuff)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [25]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="", promptsuff ="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt, promptsuff=promptsuff)
    return loss

In [26]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=1, bert=True, tokenizer=None)
print(loss)



Doc: Ogogo, 27, has resumed training as he continues his recovery from a knee operation in early February
Prediction: Ogogo, 27, has resumed training as he continues his recovery from a knee operation in early February. Ogogo suffered medial knee ligament damage against Scunthorpe on 11 February but had still played in enough games this season to trigger a contract extension.
Target: Shrewsbury Town midfielder Abu Ogogo could make a surprise return before the end of the season.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.in

{'precision': tensor([0.7852]), 'recall': tensor([0.8065]), 'f1': tensor([0.7957])}
(tensor(0.7852), tensor(0.8065), tensor(0.7957))


In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

The following layers were not sharded: transformer.h.*.attn.c_proj.bias, transformer.h.*.mlp.c_proj.bias, transformer.h.*.attn.c_attn.weight, transformer.ln_f.bias, transformer.h.*.attn.c_proj.weight, lm_head.weight, transformer.h.*.ln_*.weight, transformer.h.*.mlp.c_fc.bias, transformer.h.*.ln_*.bias, transformer.h.*.mlp.c_proj.weight, transformer.h.*.mlp.c_fc.weight, transformer.h.*.attn.c_attn.bias, transformer.ln_f.weight, transformer.wte.weight, transformer.wpe.weight


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [28]:
summary = "\n###Summary\n"

In [29]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += f'{i+1}. {batch['document'][i]}\n###Summary\n{batch['summary'][i]}'
            prompt += tokenizer.eos_token
    return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

1. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors and the roof burnt out.
The category A listed building had fallen into a state of disrepair and had been described as a target for vandals, fireraisers and drug-users.
The mansion house at Kirkton was built in the middle of the 18th Century, with a number of extensions added over the years.
From the late 1960s until about 2000 it was owned by the Leonard Cheshire foundation.
Plans for a housing development at the site were rejected and earlier this month the developers sought a council review of the decision.
###Summary
A derelict historic house near Dumfries has been damaged in a fire which broke out overnight on Friday.<|endoftext|>2. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez leve

In [30]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer, prompt = prompt, promptsuff=summary))


1. Several fire appliances were called to the scene at Carnsalloch House to tackle the blaze.
One witness reported damage to the main part of the house at Kirkton, with floors and the roof burnt out.
The category A listed building had fallen into a state of disrepair and had been described as a target for vandals, fireraisers and drug-users.
The mansion house at Kirkton was built in the middle of the 18th Century, with a number of extensions added over the years.
From the late 1960s until about 2000 it was owned by the Leonard Cheshire foundation.
Plans for a housing development at the site were rejected and earlier this month the developers sought a council review of the decision.
###Summary
A derelict historic house near Dumfries has been damaged in a fire which broke out overnight on Friday.<|endoftext|>2. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez leve

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Four men, aged between 25 and 61, were arrested in Ashbourne, County Meath, at about 16:00 GMT on Tu
Prediction: A gardaí are conducting an operation to trace the four men, who are believed to be from the north of Ireland.
A spokesman for the Garda Service said the operation was "critical" and that they would be using "legalised"
Target: Cannabis with an estimated street value of 3.2m euros (£2.7m) has been seized in a joint operation by police and customs in the Republic of Ireland.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.7316]), 'recall': tensor([0.7319]), 'f1': tensor([0.7317])}
(tensor(0.7316), tensor(0.7319), tensor(0.7317))


In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.embed_tokens.weight, model.layers.*.input_layernorm.weight, model.layers.*.post_attention_layernorm.weight, lm_head.weight, model.layers.*.self_attn.q_proj.weight, model.norm.weight, model.layers.*.self_attn.k_proj.weight, model.layers.*.self_attn.v_proj.bias


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [32]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer3, prompt=prompt3, promptsuff = summary))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: Network Rail is doing work in the Chipping Sodbury and Alderton tunnels and building a new platform 
Prediction: The Network Rail is working on the Great Western Rail mainline in Chipping Sodbury and Alderton tunnels to improve the quality of the service and to ensure they are able to meet the needs of the growing number of passengers travelling between south Wales and
Target: Rail upgrade works affecting people travelling between south Wales, Bristol Parkway and London Paddington have begun.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.7427]), 'recall': tensor([0.7944]), 'f1': tensor([0.7677])}
(tensor(0.7427), tensor(0.7944), tensor(0.7677))


In [33]:
prompt4 = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
print(
    evaluate_model(
        model3,
        test_loader,
        device="cuda",
        eval_iter=1,
        bert=False,
        tokenizer=tokenizer3,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: The 28-year-old Croatia international will sign the deal, which has a £109m (125m euros) buyout clau
Prediction: 
Target: Barcelona midfielder Ivan Rakitic has agreed a new contract, tying him to the Spanish champions until 2021.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.2134]), 'recall': tensor([0.0594]), 'f1': tensor([0.0929])}
(tensor(0.2134), tensor(0.0594), tensor(0.0929))


In [34]:
print(
    evaluate_model(
        model2,
        test_loader,
        device="cuda",
        eval_iter=2,
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Isobel Parker, 23, who was known as Becky, died at a property in Booth Place, Burnham-on-Crouch, on 
Prediction: 
The couple had been married for 17 years.
A post-mortem examination found Miss Parker had been strangled with a ligature.
She had died at the home of her mother, who had been in a relationship with her husband.

Target: The family of a pregnant woman who was strangled to death in Essex said she was "an absolute angel".


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: A Porthcawl RNLI crew with a medic and the coastguard search and rescue helicopter were sent to Sker
Prediction: 
She was taken to the Royal Norfolk Hospital in Norfolk where she was evaluated for a concussion and concussions.
She was transferred to the Royal Norfolk Hospital in Cardiff where she was treated for a concussion and concussions.
She was flown to the
Target: A woman has been airlifted to hospital after falling from a horse on a Bridgend county beach.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.8024, 0.6930]), 'recall': tensor([0.8107, 0.7252]), 'f1': tensor([0.8065, 0.7088])}
(tensor(0.7477), tensor(0.7679), tensor(0.7576))


In [35]:
import torch

print(torch.cuda.is_available())

True


In [36]:
# !pip install -U bitsandbytes

In [37]:
from unsloth import FastLanguageModel

ft_model, ft_tokenzier = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=4096, load_in_4bit=False
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipython-input-4133930992.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: model.embed_tokens.weight, model.layers.*.input_layernorm.weight, model.layers.*.post_attention_layernorm.weight, lm_head.weight, model.layers.*.self_attn.q_proj.weight, model.norm.weight, model.layers.*.self_attn.k_proj.weight, model.layers.*.self_attn.v_proj.bias


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Qwen/Qwen1.5-0.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [38]:
ft_ds = ds


def merge_columns(example):
    example["text"] = (
        f"{example['document']}\n### Summary\n{example['summary']}"
    )
    return example

ft_ds = ft_ds.map(merge_columns)

Map:   0%|          | 0/32053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1746 [00:00<?, ? examples/s]

Map:   0%|          | 0/1727 [00:00<?, ? examples/s]

In [42]:
print(ft_ds)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id', 'text'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id', 'text'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id', 'text'],
        num_rows: 1727
    })
})


In [39]:
from trl import SFTTrainer, SFTConfig
ft_model = FastLanguageModel.get_peft_model(
    ft_model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=32,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=1234,
    use_rslora = False,
    loftq_config = None,

)
trainer = SFTTrainer(
    model=ft_model,
    train_dataset=ft_ds["train"],
    eval_dataset=ft_ds["validation"],
    tokenizer=ft_tokenzier,
    args=SFTConfig(
        max_seq_length=4096,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        # num_train_epochs=1,
        max_steps=200,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        eval_strategy="steps",
        eval_steps=200,
    ),
)
trainer.train()

Unsloth 2025.11.3 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/32053 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1746 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32,053 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 7,569,408 of 471,557,120 (1.61% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tom-p-smolen (tom-p-smolen-politechnika-warszawska) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss
200,2.396300,2.503951


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=200, training_loss=2.5159029030799864, metrics={'train_runtime': 469.1696, 'train_samples_per_second': 3.41, 'train_steps_per_second': 0.426, 'total_flos': 480654327214080.0, 'train_loss': 2.5159029030799864, 'epoch': 0.04991576714294628})

In [43]:
evaluate_model(ft_model, test_loader, 'cuda', 10, bert=False, tokenizer = ft_tokenzier, promptsuff = "\n### Summary\n")

Doc: The 22-year-old played for Yeovil in the 2012-13 season and briefly last term, but has been studying
Prediction: A former Yeovil Town player has been chosen to represent the Welsh national team.
Target: Women's Super League One side Yeovil Town Ladies have re-signed goalkeeper Beth Howard after she graduated from university in the United States.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


Doc: He is the 19-year-old son of Jussi Jaaskelainen, who made 529 appearances in 15 seasons at Bolton, a
Prediction: Former Bolton striker Gary Neville has signed a two-year deal at Crewe Alexandra.
Target: Crewe Alexandra have signed Finnish goalkeeper Will Jaaskelainen on a one-year-deal after leaving Bolton.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


Doc: Canadian-Egyptian Mohamed Fahmy, Egyptian Baher Mohamed and Australian Peter Greste were sentenced t
Prediction: Three journalists who were jailed for allegedly spreading false news in Egypt have been granted bail to await a retrial.
Target: An Egyptian court has again delayed a verdict in the retrial of three al-Jazeera journalists convicted of aiding the banned Muslim Brotherhood.


...

The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


Doc: The supporters had been on a stadium tour but managed to cut themselves off from the remainder of th
Prediction: A group of supporters have been arrested after they were found with a fake explosive bomb.
Target: Two Manchester United fans spent Friday night undetected in an Old Trafford toilet in an attempt to see Saturday's Premier League game against Arsenal.


The following layers were not sharded: encoder.layer.*.attention.output.LayerNorm.bias, encoder.rel_embeddings.weight, embeddings.LayerNorm.weight, embeddings.word_embeddings.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.LayerNorm.bias, encoder.layer.*.intermediate.dense.bias, encoder.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.attention.self.key_proj.weight, encoder.LayerNorm.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight


{'precision': tensor([0.8345, 0.8413, 0.8593, 0.8423, 0.7855, 0.8033, 0.8185, 0.8597, 0.8379,
        0.8074]), 'recall': tensor([0.7974, 0.8129, 0.8362, 0.8359, 0.7562, 0.8181, 0.8193, 0.8389, 0.8337,
        0.7740]), 'f1': tensor([0.8156, 0.8269, 0.8476, 0.8391, 0.7705, 0.8107, 0.8189, 0.8492, 0.8358,
        0.7904])}


(tensor(0.8290), tensor(0.8123), tensor(0.8205))

In [52]:
ft_model.save_pretrained('./drive/MyDrive/models/ft')

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
